# Metaverse Avatar

In [20]:
pip install mesa

Note: you may need to restart the kernel to use updated packages.


In [21]:
from mesa import Agent,Model

Here we define the avatar with all its functionalities

In [22]:
class Agent_Avatar(Agent):
    subjects = {}
    preferences = {}

    # First Deliverable
    def __init__(self):
        print('Hello human, I am your avatar, you can configure me to be as similar to you as possible, my creators are Anthony Garcia, Sebastian Arboleda and Katherin Valencia.')

    # Second Deliverable
    def setScheduleOptions(self, subjects, preferences):
        self.subjects = subjects
        self.preferences = preferences

    def getBetterSchedule(self):
        independent, schedule = self.__isIndependent(list(self.subjects.keys()))

        print("============== BETTER WAY TO CHOOSE SUBJECTS ==============")
        score = 0
        for h in schedule:
            subject, dayTime = h
            score += dayTime[1]
            print('Subject: %s, Schedule: %s, Score: %s' % (subject, str(dayTime[0]), dayTime[1]))
        print('TOTAL SCORE: %i' % score)

    def __isIndependent(self, subjs):
        '''Determines if a set of subjects is independent'''
        subjs = self.__orderSubjects(subjs)
        if len(subjs) == 0:
            return False, []
        score, independent = self.__overlap(subjs)
                        
        return True, independent

    def __orderSubjects(self, subjs):
        ordered = []
        for sub in subjs:
            ordered.append([
                sub, self.preferences['subjects'][sub]
            ])
            ordered = sorted(ordered, key=lambda d: d[1], reverse=True)
        subs = []
        for i in ordered:
            subs.append(i[0])
        return subs

    def __overlap(self, subjs):
        return self.__recursiveOverlap([], subjs)

    def __recursiveOverlap(self, actualSchedule, subjs):
        if len(subjs) == 0:
            return 0, actualSchedule
        actualSbj = subjs[0]
        currentScore = 0
        actualScheduleCopy = actualSchedule.copy()

        for i in self.subjects[actualSbj]:
            if not self.__overlapClass(actualSchedule, i):
                newScore = self.__getScore(actualSbj, i)
                schedule = actualScheduleCopy.copy()
                schedule.append((actualSbj, (i, newScore)))
                score, schedule = self.__recursiveOverlap(schedule, subjs[1:])
                if score + newScore > currentScore:
                    actualSchedule = actualScheduleCopy.copy()
                    currentScore = score + newScore
                    actualSchedule = schedule
            score, schedule = self.__recursiveOverlap(actualSchedule, subjs[1:])
            if score > currentScore:
                actualSchedule = actualScheduleCopy.copy()
                currentScore = score
                actualSchedule = schedule
        return currentScore, actualSchedule


    def __overlapClass(self, schedulesList, actualSchedule):
        overlap = True
        for h2 in actualSchedule:
            currentOverlap = False
            for schedule in schedulesList:
                classes = schedule[1][0]
                for h1 in classes:
                    if h1[0] == h2[0]:
                        # Which one starts before
                        if h1[1] <= h2[1]:
                            first, second = h1, h2  
                        else:
                            first, second = h2, h1
                        currentOverlap = currentOverlap or first[2] > second[1]
            overlap = overlap and currentOverlap
        return overlap

    def __getScore(self, subj, schedule):
        subjScore = self.preferences['subjects'][subj]
        scheduleScore = 0.0
        for i in schedule: 
            scheduleScore += self.preferences['days'][i[0]]
        return scheduleScore/len(schedule) + subjScore


Test data for Second Deliverable

In [23]:
subjects = {
    "AI introduction": [[("Monday","0830","1130")], [("Friday", "0830","1130")]] ,
    "Modern SW development processes":  [[("Friday","1800","2100"), ("Saturday", "0800","1100")]],
    "Projects management":  [[("Monday","0600","0900")]],
    "UX Design": [[("Wednesday","1800","2100")]],
    "Karate":  [[("Monday","1630","1800")], [("Wednesday", "1630","1800")], [("Friday", "1630","1800")]],
    "Rest": [[("Friday","0400","0930")]]
}


preferences = {
    "subjects": {
        "AI introduction": 3,
        "Modern SW development processes": 2,
        "Projects management": 1,
        "UX Design": 1,
        "Karate": 2,
        "Rest": 3,
    },
    "days" : {
        "Monday": 2,
        "Tuesday": 4,
        "Wednesday": 4,
        "Thursday": 4,
        "Friday": 3,
        "Saturday": 1,
    },
}

Here we define the Model

In [24]:
class Model_Avatar(Model):
    agent = {}

    def __init__(self):
        self.agent = Agent_Avatar()

    def secondDeliverable(self):
        self.agent.setScheduleOptions(subjects, preferences)
        self.agent.getBetterSchedule()

## Call for Hello World (1st Deliverable)

In [25]:
Avatar = Model_Avatar()

Hello human, I am your avatar, you can configure me to be as similar to you as possible, my creators are Anthony Garcia, Sebastian Arboleda and Katherin Valencia.


## Call for the Search Problem (2nd Deliverable)


In [26]:
Avatar.secondDeliverable()

============== BETTER WAY TO CHOOSE SUBJECTS ==============
Subject: AI introduction, Schedule: [('Monday', '0830', '1130')], Score: 5.0
Subject: Rest, Schedule: [('Friday', '0400', '0930')], Score: 6.0
Subject: Modern SW development processes, Schedule: [('Friday', '1800', '2100'), ('Saturday', '0800', '1100')], Score: 4.0
Subject: Karate, Schedule: [('Wednesday', '1630', '1800')], Score: 6.0
Subject: UX Design, Schedule: [('Wednesday', '1800', '2100')], Score: 5.0
TOTAL SCORE: 26


In [27]:
%pip install pyswip

Note: you may need to restart the kernel to use updated packages.


# 3rd Deliverable

In [28]:
from pyswip import Prolog

In [29]:
prolog = Prolog()
prolog.consult("prolog.pl")

Here is defined the First chronicle

In [30]:
def isUnderstanding(subject):
    query_args = "understood_class(%s,X)" % (subject.lower().replace(" ","_"))
    query = list(prolog.query(query_args))
    count = 0
    for res in query:
        count+= res["X"]
    return (count/len(query)) >= 0.8

subjects = [
    "AI Introduction",
    "Modern SW development processes",
    "Projects management",
    "UX Design"
]

def calcUnderstanding():
    for i in subjects:
        print("Is understanding %s?" % (i))
        print(isUnderstanding(i))

Here is defined the Second Chronicle

In [31]:
def getAvailableTime():
    query = prolog.query("week(X,Y)")
    freeTime = {}
    for res in query:
        freeTime[res["X"]] = int(res["Y"])
    return freeTime

def printAvailableTime():
    for week, time in getAvailableTime().items():
        print("Available time for week %s: %i" % (week, time))

def  addHomework(name, week, time):
    print("Adding Homework %s..." % (name))
    query = prolog.query("add_homework(%s,%s,%s,X)" % (name.lower().replace(" ","_"), week.lower(), str(time)))
    for res in query:
        print("AvailableTime: "+ str(res["X"]))

def  completeHomework(name, time):
    print("Completed Homework %s..." % (name))
    query = prolog.query("complete_homework(%s,%s,X)" % (name.lower().replace(" ","_"), str(time)))
    for res in query:
        print("AvailableTime: "+ str(res["X"]))

def listPendingHomeworks(week):
    print("Pending Homeworks for week %s:" % (week))
    query = prolog.query("homework(X,%s,Y)" % (week.lower()))
    for res in query:
        print("Homework: %s, Time: %s" % (res["X"],res["Y"]))

def addHomeworks():
    addHomework("Reading Forum", "one", 3)
    addHomework("Task 1", "one", 3)
    addHomework("Task 2", "one", 6)
    addHomework("Task 3", "one", 10)
    addHomework("Task 4", "one", 8)
    addHomework("UX Diagram", "two", 4)
    addHomework("Presentations ", "two", 3)
    addHomework("PMDS Test", "three", 10)
    addHomework("AI Agent", "four", 3)
    addHomework("Task 5", "four", 10)
    addHomework("Task 6", "four", 5)
    addHomework("Task 7", "four", 4)
    addHomework("Task 8", "four", 9)
    addHomework("Task 9", "four", 10)
    addHomework("Task 10", "four", 10)
    addHomework("Delivery 3", "four", 8)

In [32]:
addHomeworks()
printAvailableTime()

Adding Homework Reading Forum...
AvailableTime: 103
AvailableTime: 73
Adding Homework Task 1...
AvailableTime: 100
AvailableTime: 70
Adding Homework Task 2...
AvailableTime: 94
AvailableTime: 64
Adding Homework Task 3...
AvailableTime: 84
AvailableTime: 54
Adding Homework Task 4...
AvailableTime: 76
AvailableTime: 46
Adding Homework UX Diagram...
AvailableTime: 102
AvailableTime: 95
Adding Homework Presentations ...
AvailableTime: 99
AvailableTime: 92
Adding Homework PMDS Test...
AvailableTime: 87
AvailableTime: 77
AvailableTime: 77
Adding Homework AI Agent...
AvailableTime: 94
AvailableTime: 30
Adding Homework Task 5...
AvailableTime: 84
AvailableTime: 20
Adding Homework Task 6...
AvailableTime: 79
AvailableTime: 15
Adding Homework Task 7...
AvailableTime: 75
AvailableTime: 11
Adding Homework Task 8...
AvailableTime: 66
AvailableTime: 2
Adding Homework Task 9...
AvailableTime: 56
AvailableTime: -8
Adding Homework Task 10...
AvailableTime: 46
AvailableTime: -18
Adding Homework Delivery

In [33]:
listPendingHomeworks("four")
completeHomework("AI Agent", 8)
listPendingHomeworks("four")

Pending Homeworks for week four:
Homework: task_5, Time: 10
Homework: task_6, Time: 5
Homework: task_7, Time: 4
Homework: task_8, Time: 9
Homework: task_9, Time: 10
Homework: task_10, Time: 10
Homework: delivery_3, Time: 8
Homework: ai_agent, Time: 3
Homework: ai_agent, Time: 3
Homework: task_5, Time: 10
Homework: task_5, Time: 10
Homework: task_6, Time: 5
Homework: task_6, Time: 5
Homework: task_7, Time: 4
Homework: task_7, Time: 4
Homework: task_8, Time: 9
Homework: task_8, Time: 9
Homework: task_9, Time: 10
Homework: task_9, Time: 10
Homework: task_10, Time: 10
Homework: task_10, Time: 10
Homework: delivery_3, Time: 8
Homework: delivery_3, Time: 8
Completed Homework AI Agent...
AvailableTime: 33
Pending Homeworks for week four:
Homework: task_5, Time: 10
Homework: task_6, Time: 5
Homework: task_7, Time: 4
Homework: task_8, Time: 9
Homework: task_9, Time: 10
Homework: task_10, Time: 10
Homework: delivery_3, Time: 8
Homework: task_5, Time: 10
Homework: task_5, Time: 10
Homework: task_

In [34]:
activities = {
    "Karate": 16,
    "Reading": 10,
}

def hasEnoughFreeTime():
    neededTime = 0
    for act in activities.values():
        neededTime += act
    weeklyTime = {}
    for week,time in getAvailableTime().items():
        condition = time >= neededTime
        weeklyTime[week] = condition
    return weeklyTime

Here is defined the Third Chronicle

In [35]:
weeks = ["one","two","three","four","five","six","seven"]

def isHealthy(week):
    weeklyTime = hasEnoughFreeTime()
    query = list(prolog.query("weekly_health(%s,X)" % (week)))
    score = 0
    for res in query:
        score += res["X"]
    return (score) > 0 and weeklyTime[week]

def calcWeeklyHealth():
    for i in weeks:
        print ("Is healthy on week %s?..." % (i))
        print(isHealthy(i))

Call For first result

In [36]:
calcUnderstanding()

Is understanding AI Introduction?
True
Is understanding Modern SW development processes?
False
Is understanding Projects management?
True
Is understanding UX Design?
True


Call for second result

In [37]:
hasEnoughFreeTime()

{'five': True,
 'six': True,
 'seven': True,
 'one': True,
 'two': True,
 'three': True,
 'four': True}

Call for third result

In [38]:
calcWeeklyHealth()

Is healthy on week one?...
True
Is healthy on week two?...
True
Is healthy on week three?...
True
Is healthy on week four?...
False
Is healthy on week five?...
True
Is healthy on week six?...
False
Is healthy on week seven?...
True
